# Comparison of SuSiE and Singleton MvSuSiE

In [14]:
library(dplyr)
library(readr)
library(tibble)
library(ggplot2)
library(purrr)
library(tidyr)
library("ggpubr")
library(susieR)

## Generation of Singleton Prior

In [4]:
Prior = readRDS("~/Project//one_gene/data/GTEx_V8_strong_z.ed_8tissue.rds")
tissue = Prior$U$Artery_Tibial%>%colnames()
mtr = matrix(0, nrow = 8, ncol = 8,dimnames = list(tissue,tissue))
mtr[colnames(mtr) == "Whole_Blood",colnames(mtr) == "Whole_Blood"] = 1
mtr%>%saveRDS("~/Project//one_gene/data/GTEx_V8_strong_z.ed_Whole_Blood.rds")

,Artery_Tibial,Liver,Lung,Muscle_Skeletal,Spleen,Stomach,Thyroid,Whole_Blood
Artery_Tibial,0,0,0,0,0,0,0,0
Liver,0,0,0,0,0,0,0,0
Lung,0,0,0,0,0,0,0,0
Muscle_Skeletal,0,0,0,0,0,0,0,0
Spleen,0,0,0,0,0,0,0,0
Stomach,0,0,0,0,0,0,0,0
Thyroid,0,0,0,0,0,0,0,0
Whole_Blood,0,0,0,0,0,0,0,1


## Uni SuSiE

In [ ]:
for tissue in `echo Whole_Blood`; do
nohup sos run ~/GIT/neuro-twas/workflow/uni_susie.ipynb susie_cv  \
  --molecular-pheno  /home/hs3163/Project/one_gene/GTEx_V8.txt  \
  --wd "/mnt/mfs/statgen/neuro-twas/JAZF1_uni_non/$tissue/" \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/one_gene/GTEx_V8.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --cv_times 100 \
  --job_size 1 \
  --MAF 0 \
  -s build \
  -J 6 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml &
done

## Running MV_Susie singleton

In [ ]:
for tissue in `echo Whole_Blood`; do
nohup sos run /home/hs3163/GIT/neuro-twas/workflow/mv_susie.ipynb mv_susie_cv \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/JAZF1/mole_pheno_ls" \
--region_list "/mnt/mfs/statgen/neuro-twas/JAZF1//region_list" \
--wd /mnt/mfs/statgen/neuro-twas/JAZF1_diag/$tissue/ \
--name_prefix "GTEx_V8" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--mixture_prior ~/Project//one_gene/data/GTEx_V8_strong_z.ed_$tissue.rds \
--cv_times 100 \
--MAF 0 \
--extract_snp "/mnt/mfs/statgen/neuro-twas/JAZF1_uni_non//Whole_Blood/whole_blood_snps" \
-J 200 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml   &
done

## Loading data

In [11]:
whole_blood_mv = attach("/mnt/mfs/statgen/neuro-twas/JAZF1_diag/Whole_Blood/result/GTEx_V8.ENSG00000153814.mv_susie.model.RData")$m_ENSG00000153814

The following object is masked from file:/mnt/mfs/statgen/neuro-twas/JAZF1_diag/Whole_Blood/result/GTEx_V8.ENSG00000153814.mv_susie.model.RData (pos = 5):

    m_ENSG00000153814


The following object is masked from file:/mnt/mfs/statgen/neuro-twas/JAZF1_diag/Whole_Blood/result/GTEx_V8.ENSG00000153814.mv_susie.model.RData (pos = 6):

    m_ENSG00000153814




In [12]:
whole_blood_susie = attach("/mnt/mfs//statgen/neuro-twas/JAZF1_uni_non/Whole_Blood/susie/GTEx_V8.ENSG00000153814.uni_weight.cv.RData")$fitted1

The following object is masked from file:/mnt/mfs//statgen/neuro-twas/JAZF1_uni_non/Whole_Blood/susie/GTEx_V8.ENSG00000153814.uni_weight.cv.RData (pos = 4):

    fitted1


The following object is masked from file:/mnt/mfs//statgen/neuro-twas/JAZF1_uni_non/Whole_Blood/susie/GTEx_V8.ENSG00000153814.uni_weight.cv.RData (pos = 5):

    fitted1




In [21]:
lm

## Comparison of PIP

In [25]:
pip = cbind(whole_blood_mv$pip,whole_blood_susie$pip)%>%as_tibble
t.test(pip[,1],pip[,2])

Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
Using compatibility `.name_repair`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”



	Welch Two Sample t-test

data:  pip[, 1] and pip[, 2]
t = -6.3216, df = 58471, p-value = 2.608e-10
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.0004027081 -0.0002120902
sample estimates:
   mean of x    mean of y 
3.419973e-05 3.415989e-04 


In [30]:
lm(V1~V2,data = pip )%>%summary


Call:
lm(formula = V1 ~ V2, data = pip)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.080850  0.000048  0.000141  0.000167  0.022094 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.999e-04  5.092e-06  -58.91   <2e-16 ***
V2           9.782e-01  8.599e-04 1137.52   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.0008693 on 29238 degrees of freedom
Multiple R-squared:  0.9779,	Adjusted R-squared:  0.9779 
F-statistic: 1.294e+06 on 1 and 29238 DF,  p-value: < 2.2e-16


## Comparison of Coef

In [31]:
coef = cbind(whole_blood_mv$coef[,8],whole_blood_susie%>%coef.susie)%>%as_tibble

In [32]:
t.test(coef[,1],coef[,2])


	Welch Two Sample t-test

data:  coef[, 1] and coef[, 2]
t = 1.6084, df = 58449, p-value = 0.1077
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -4.712598e-06  4.783076e-05
sample estimates:
    mean of x     mean of y 
 9.443668e-06 -1.211541e-05 


In [33]:
lm(V1~V2,data = coef )%>%summary


Call:
lm(formula = V1 ~ V2, data = coef)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0060873 -0.0000026  0.0000016  0.0000052  0.0109162 

Coefficients:
              Estimate Std. Error   t value Pr(>|t|)    
(Intercept) -2.370e-06  6.074e-07    -3.902 9.57e-05 ***
V2          -9.751e-01  3.705e-04 -2631.618  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.0001039 on 29239 degrees of freedom
Multiple R-squared:  0.9958,	Adjusted R-squared:  0.9958 
F-statistic: 6.925e+06 on 1 and 29239 DF,  p-value: < 2.2e-16


## Comparison of fitted result

Warning message in cbind(whole_blood_mv$fitted[, 8], whole_blood_susie$fitted):
“number of rows of result is not a multiple of vector length (arg 2)”


In [54]:
samples = read_delim("/mnt/mfs/statgen/neuro-twas/JAZF1_diag/Whole_Blood//cache//GTEx_V8.ENSG00000153814.merged.fam",col_names = F,delim = " ")


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_character(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double()
)




In [58]:
samples = samples%>%mutate(X1 = map_chr(X1,~paste(c(.x,.x),collapse = ":")))

In [71]:
samples_uni = whole_blood_susie$fitted%>%names()%>%as_tibble%>%select(X1 = value )

In [75]:
overlap = which(samples$X1  %in% samples_uni$X1)

In [76]:
fitted = cbind(whole_blood_mv$fitted[overlap,8],whole_blood_susie$fitted)%>%as_tibble

In [87]:
t.test(fitted[,1],fitted[,2])


	Welch Two Sample t-test

data:  fitted[, 1] and fitted[, 2]
t = 0.0030061, df = 1316.6, p-value = 0.9976
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.03097463  0.03106970
sample estimates:
    mean of x     mean of y 
 4.753643e-05 -3.034496e-15 


In [88]:
lm(V1~V2,data = fitted )%>%summary


Call:
lm(formula = V1 ~ V2, data = fitted)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.46153 -0.20646  0.07307  0.17785  0.65266 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 4.754e-05  9.971e-03   0.005    0.996    
V2          2.640e-01  3.247e-02   8.131 2.07e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2581 on 668 degrees of freedom
Multiple R-squared:  0.09006,	Adjusted R-squared:  0.0887 
F-statistic: 66.12 on 1 and 668 DF,  p-value: 2.067e-15


In [90]:
sum(fitted[,1])

[1] 0.03184941

In [89]:
sum(fitted[,2])

[1] -2.033096e-12